In [1]:
!wandb disabled


W&B disabled.


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [3]:
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, ClassLabel, load_metric
import numpy as np
import torch

In [4]:
# Load the tokenizer and model
model_name = "google/bigbird-roberta-base"
tokenizer = BigBirdTokenizer.from_pretrained(model_name)
model = BigBirdForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Load dataset
import pandas as pd
from datasets import Dataset

# Load the JSON file into a pandas DataFrame
df = pd.read_json("uoa-11-trainset-cleaned.json")

# Convert the pandas DataFrame to a datasets Dataset
dataset = Dataset.from_pandas(df)

In [6]:
# Label adjustment: convert 2, 3, 4 to 0, 1, 2 for model
def preprocess_labels(example):
    example["label"] = int(example["label"]) - 2
    return example

dataset = dataset.map(preprocess_labels)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [7]:
# Tokenization
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=4096,
        return_tensors="pt"
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [8]:
# Training args
training_args = TrainingArguments(
    output_dir="./bigbird-ref",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=5,
    save_steps=10,
    save_total_limit=2,
    report_to="none",              # ensures output is shown in notebook
    disable_tqdm=False,            # enables tqdm logging bar
    logging_strategy="steps",      # explicitly logs by steps
)

In [ ]:
# Use the full dataset for training without splitting
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train
trainer.train()

/tmp/ipython-input-9-1630225373.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Save model
trainer.save_model("uoa_11_model")
tokenizer.save_pretrained("uoa_11_model")

# Print final training logs
for log in trainer.state.log_history:
    print(log)
